# Advanced Topics: Top-K and Self Joins

## Setup

In [1]:
import os
import ibis

ibis.options.interactive = True

connection = ibis.sqlite.connect(os.path.join('data', 'geography.db'))

## "Top-K" Filtering


A common analytical pattern involves subsetting based on some method of ranking. For example, "the 5 most frequently occurring widgets in a dataset". By choosing the right metric, you can obtain the most important or least important items from some dimension, for some definition of important.

To carry out the pattern by hand involves the following

- Choose a ranking metric
- Aggregate, computing the ranking metric, by the target dimension
- Order by the ranking metric and take the highest K values
- Use those values as a set filter (either with `semi_join` or `isin`) in your next query

For example, let's look at the TPC-H tables and find the 5 or 10 customers who placed the most orders over their lifetime:

In [3]:
countries = connection.table('countries')
countries

    iso_alpha2 iso_alpha3  iso_numeric fips                  name  \
0           AD        AND           20   AN               Andorra   
1           AE        ARE          784   AE  United Arab Emirates   
2           AF        AFG            4   AF           Afghanistan   
3           AG        ATG           28   AC   Antigua and Barbuda   
4           AI        AIA          660   AV              Anguilla   
..         ...        ...          ...  ...                   ...   
247         YE        YEM          887   YM                 Yemen   
248         YT        MYT          175   MF               Mayotte   
249         ZA        ZAF          710   SF          South Africa   
250         ZM        ZMB          894   ZA                Zambia   
251         ZW        ZWE          716   ZI              Zimbabwe   

              capital   area_km2  population continent  
0    Andorra la Vella      468.0       84000        EU  
1           Abu Dhabi    82880.0     4975593        AS  


In [2]:
countries = connection.table('countries')

country_size = (
    ibis.case()
    .when(countries.population > 25_000_000, 'big')
    .when(countries.population < 5_000_000, 'small')
    .else_('medium')
    .end()
    .name('size')
)

countries.group_by(country_size).size()

     size  count
0     big     46
1  medium     70
2   small    136

In [17]:
continents_with_many_countries = (
    countries.group_by('continent').size().sort_by(('count', False)).limit(3)
)
continents_with_many_countries

  continent  count
0        AF     58
1        EU     54
2        AS     51

In [25]:
countries[countries.continent.isin(continents_with_many_countries.continent)]

    iso_alpha2 iso_alpha3  iso_numeric fips                  name  \
0           AD        AND           20   AN               Andorra   
1           AE        ARE          784   AE  United Arab Emirates   
2           AF        AFG            4   AF           Afghanistan   
3           AL        ALB            8   AL               Albania   
4           AM        ARM           51   AM               Armenia   
..         ...        ...          ...  ...                   ...   
158         YE        YEM          887   YM                 Yemen   
159         YT        MYT          175   MF               Mayotte   
160         ZA        ZAF          710   SF          South Africa   
161         ZM        ZMB          894   ZA                Zambia   
162         ZW        ZWE          716   ZI              Zimbabwe   

              capital   area_km2  population continent  
0    Andorra la Vella      468.0       84000        EU  
1           Abu Dhabi    82880.0     4975593        AS  


Now, we could use these customer keys as a filter in some other analysis:

In [5]:
# Among the top 3 continents with more countries, what's the number of countries by size?
analysis = (
    countries[
        countries.continent.isin(continents_with_many_countries.continent)
    ]
    .group_by(country_size)
    .size()
)
analysis

     size  count
0     big     38
1  medium     57
2   small     68

This is such a common pattern that Ibis supports a high level primitive `topk` operation, which can be used immediately as a filter:

In [24]:
countries

    iso_alpha2 iso_alpha3  iso_numeric fips                  name  \
0           AD        AND           20   AN               Andorra   
1           AE        ARE          784   AE  United Arab Emirates   
2           AF        AFG            4   AF           Afghanistan   
3           AG        ATG           28   AC   Antigua and Barbuda   
4           AI        AIA          660   AV              Anguilla   
..         ...        ...          ...  ...                   ...   
247         YE        YEM          887   YM                 Yemen   
248         YT        MYT          175   MF               Mayotte   
249         ZA        ZAF          710   SF          South Africa   
250         ZM        ZMB          894   ZA                Zambia   
251         ZW        ZWE          716   ZI              Zimbabwe   

              capital   area_km2  population continent  
0    Andorra la Vella      468.0       84000        EU  
1           Abu Dhabi    82880.0     4975593        AS  


In [23]:
countries[countries.continent.topk(3)]

     iso_alpha2 iso_alpha3  iso_numeric fips                  name  \
0            AD        AND           20   AN               Andorra   
1            AE        ARE          784   AE  United Arab Emirates   
2            AF        AFG            4   AF           Afghanistan   
3            AG        ATG           28   AC   Antigua and Barbuda   
4            AI        AIA          660   AV              Anguilla   
...         ...        ...          ...  ...                   ...   
9995         NL        NLD          528   NL           Netherlands   
9996         NO        NOR          578   NO                Norway   
9997         NP        NPL          524   NP                 Nepal   
9998         NR        NRU          520   NR                 Nauru   
9999         NU        NIU          570   NE                  Niue   

               capital  area_km2  population continent  
0     Andorra la Vella     468.0       84000        EU  
1            Abu Dhabi   82880.0     4975593 

In [16]:
# continents_with_many_countries = countries.continent.topk(3)
countries[continents_with_many_countries].group_by(country_size).size()
countries[continents_with_many_countries]
countries

RelationError: The expression   continent  count
0        AF     58
1        EU     54
2        AS     51 does not fully originate from dependencies of the table expression.

This goes a little further. Suppose now we want to rank customers by their total spending instead of the number of orders, perhaps a more meaningful metric:

In [ ]:
total_spend = orders.o_totalprice.sum().name('total')
top_spenders = (
    orders.group_by('o_custkey')
    .aggregate(total_spend)
    .sort_by(('total', False))
    .limit(5)
)
top_spenders

To use another metric, just pass it to the `by` argument in `topk`:

In [ ]:
top_spenders = orders.o_custkey.topk(5, by=total_spend)
orders[top_spenders].group_by('o_orderstatus').size()

## Self joins


If you're a relational data guru, you may have wondered how it's possible to join tables with themselves, because joins clauses involve column references back to the original table.

Consider the SQL

```sql
    SELECT t1.key, sum(t1.value - t2.value) AS metric
    FROM my_table t1
      JOIN my_table t2
        ON t1.key = t2.subkey
    GROUP BY 1
```
    
Here, we have an unambiguous way to refer to each of the tables through aliasing.

Let's consider the TPC-H database, and support we want to compute year-over-year change in total order amounts by region using joins.

In [ ]:
region = con.table('tpch_region')
nation = con.table('tpch_nation')
customer = con.table('tpch_customer')
orders = con.table('tpch_orders')

orders.limit(5)

First, let's join all the things and select the fields we care about:

In [ ]:
fields_of_interest = [
    region.r_name.name('region'),
    nation.n_name.name('nation'),
    orders.o_totalprice.name('amount'),
    orders.o_orderdate.cast('timestamp').name('odate'),  # these are strings
]

joined_all = (
    region.join(nation, region.r_regionkey == nation.n_regionkey)
    .join(customer, customer.c_nationkey == nation.n_nationkey)
    .join(orders, orders.o_custkey == customer.c_custkey)[fields_of_interest]
)

Okay, great, let's have a look:

In [ ]:
joined_all.limit(5)

Sweet, now let's aggregate by year and region:

In [ ]:
year = joined_all.odate.year().name('year')

total = joined_all.amount.sum().cast('double').name('total')

annual_amounts = joined_all.group_by(['region', year]).aggregate(total)
annual_amounts

Looking good so far. Now, we need to join this table on itself, by subtracting 1 from one of the year columns.

We do this by creating a "joinable" view of a table that is considered a distinct object within Ibis. To do this, use the `view` function:

In [ ]:
current = annual_amounts
prior = annual_amounts.view()

yoy_change = (current.total - prior.total).name('yoy_change')

results = current.join(
    prior,
    ((current.region == prior.region) & (current.year == (prior.year - 1))),
)[current.region, current.year, yoy_change]
df = results.execute()

In [ ]:
df['yoy_pretty'] = df.yoy_change.map(lambda x: '$%.2fmm' % (x / 1000000.0))
df

If you're being fastidious and want to consider the first year occurring in the dataset for each region to have 0 for the prior year, you will instead need to do an outer join and treat nulls in the prior side of the join as zero:

In [ ]:
yoy_change = (current.total - prior.total.zeroifnull()).name('yoy_change')
results = current.outer_join(
    prior,
    ((current.region == prior.region) & (current.year == (prior.year - 1))),
)[
    current.region,
    current.year,
    current.total,
    prior.total.zeroifnull().name('prior_total'),
    yoy_change,
]

results.limit(10)

In [2]:
gdp = connection.table('gdp')
gdp

     country_code  year         value
0             ABW  1986  4.054634e+08
1             ABW  1987  4.876025e+08
2             ABW  1988  5.964236e+08
3             ABW  1989  6.953044e+08
4             ABW  1990  7.648871e+08
...           ...   ...           ...
9995          SVK  2002  3.513034e+10
9996          SVK  2003  4.681659e+10
9997          SVK  2004  5.733202e+10
9998          SVK  2005  6.278531e+10
9999          SVK  2006  7.070810e+10

[10000 rows x 3 columns]

In [ ]:
gdp_2000 = gdp[gdp.year == 2000]['country_code', gdp.value.name('gdp_2000')]
gdp_2010 = gdp[gdp.year == 2010]['country_code', gdp.value.name('gdp_2010')]


gdp_diff = gdp_2000.join(
    gdp_2010, gdp_2010.country_code == gdp_2000.country_code
)

gdp_change = (1.0 - (gdp_2010.gdp_2010 / gdp_2000.gdp_2000)).name('gdp_change')

gdp_diff[gdp_2000.country_code, gdp_change].sort_by('gdp_change')

In [3]:
gdp_2000 = gdp[gdp.year == 2000]['country_code', gdp.value.name('gdp_2000')]
gdp_2010 = gdp[gdp.year == 2010][
    'country_code', gdp.value.name('gdp_2010')
].view()


gdp_diff = gdp_2000.join(
    gdp_2010, gdp_2010.country_code == gdp_2000.country_code
)

In [5]:
gdp_diff[gdp_2000.country_code]

     country_code
0             ABW
1             AGO
2             ALB
3             AND
4             ARB
...           ...
9995          KNA
9996          KOR
9997          KWT
9998          LAC
9999          LAO

[10000 rows x 1 columns]